# Size Factors and Transport Conductance

Consider an arbitrary pore network model such as the one shown below:

<img src="https://user-images.githubusercontent.com/14086031/188179071-01413efc-ed04-4920-b9b1-d99288ced47b.png" width=400px>

For simulating transport in this network, we need to know lots of parameters such as pore sizes, physical properties, etc, but ultimately it comes down to knowing the conductance of the "conduits" in the network. For example, consider simulating Fickian diffusion in a pore network model under steady state conditions. Writing a mass balance around an arbitrary pore $i$, we have:

$$
\sum_k^{Nb^i} m_{ik} = \sum_k^{Nb^i} g_{ik} (c_i - c_k) = 0
$$

where $Nb^i$ is the number of neighboring pores to pore $i$, $m_{ik}$ is the mass flow rate from pore $i$ to pore $k$, and $g_{ik}$ is the overall conductance of the conduit containing pores $i$ and $k$, and the connecting throat $ik$. Writing the mass balance for each pore, we get a linear system of $N_p$ equations (where $N_p$ is the number of pores in the network), which we can solve for the pore concentrations using a linear solver.

## Size Factors vs. Shape Factors

Consider a conduit in a pore network model, as shown by the following figure (we assumed spherical pores and cylindrical throats, but this is not necessary):

<img src="https://user-images.githubusercontent.com/14086031/187992821-94dee392-25a1-4fba-bce4-6ee8be8c22f2.png" width=500px>

where L is length, A is the cross-sectional area, and the subscripts $i$, $k$, and $ik$ denote pore $i$, pore $k$, and throat $ik$. Now consider diffusional mass transfer between pore $i$ and pore $k$ with a mass flow rate of $m_{ik}$, which can be described by the following equation:

$$
m_{ik} = g_{ik} \cdot (c_i - c_k)
$$

where $g_{ik}$ is the diffusive conductance of the entire conduit, and $c_i$ and $c_k$ are the concentrations in pores $i$ and $k$, respectively. Note that the conduit $ik$ consists of pore $i$, throat $ik$, and pore $j$, which are connected in series. By electrical circuit analogy, the resistance (inverse of the conductance) of each element can be related to the overall resistance by the following equation:

$$
R_{ik} = R_i^p + R_k^p + R_{ik}^t
$$

where the superscripts $p$ and $t$ denote pore and throat resistances, respectively, and $R_{ik}$ is the resistance of the entire conduit. The equation above can be rewritten in form of conductance as follows:

$$
1/g_{ik} = 1/g_i^p + 1/g_k^p + 1/g_{ik}^t
$$

where $g_i^p$ and $g_k^p$ are the pore resistances, and $g_{ik}^t$ is the throat resistance.

For simplicity, let's assume that the shape of the pores and throats are cylindrical as shown in the figure below:

<img src="https://user-images.githubusercontent.com/14086031/188184909-079977ba-d9a8-4760-99f7-4019b4e8f2a9.png" width=750px>

Based on this assumption, the diffusive conductance of each element can be expressed as:

$$
g_i^p = D \frac{\pi d_i^2/4}{\ell_i} \quad , \quad g_k^p = D \frac{\pi d_k^2/4}{\ell_k} \quad , \quad g_{ik}^t = D \frac{\pi d_{ik}^2/4}{\ell_{ik}}
$$

In reality, however, pores/throats are not cylindrical, and the diffusive conductance needs to be **corrected**. This correction factor is called the **shape factor** in the network modeling literature, as defined by the following equation:

$$
SF = \frac{\text{actual conductance}}{\text{cylindrical conductance}}
$$

:::{warning}
We used to use shape factors in the past, but they are not recommended, as me moved towards using the "size factor" instead.
:::

In OpenPNM, we used to follow this terminology, but because the shape factor depends on both geometrical and physical properties, we coined the term **size factor**, $\mathcal{S}$ to decouple the geometric part of the conductance from its physical properties dependence, such that the diffusional mass flow rate becomes:

$$
m_{ik} = \mathcal{S} \cdot D \cdot (c_i - c_k)
$$

where $D$ is the diffusivity. In other words, the size factor is the geometric part of the conductance, and so the conductance can be defined in terms of the size factor as:

$$
g = \mathcal{S} \cdot D
$$

OpenPNM has a number of built-in functions to calculate the size factor, which can be found under `openpnm.models.geometry.diffusive_size_factors`, and `openpnm.models.geometry.hydraulic_size_factors`. Let's inspect the available diffusive size factor models.

In [29]:
import openpnm as op
diffusive_size_factors = op.models.geometry.diffusive_size_factors
print("\n".join(diffusive_size_factors._funcs.__all__))

spheres_and_cylinders
circles_and_rectangles
cones_and_cylinders
intersecting_cones
hybrid_cones_and_cylinders
trapezoids_and_rectangles
hybrid_trapezoids_and_rectangles
intersecting_trapezoids
pyramids_and_cuboids
intersecting_pyramids
hybrid_pyramids_and_cuboids
cubes_and_cuboids
squares_and_rectangles
intersecting_trapezoids
ncylinders_in_series


:::{note}
Note that the diffusive size factor models can be used for any physics that involves molecular diffusion such as heat conduction, electron transport, etc, and therefore, it is not limited to *diffusive mass transport*. The hydraulic size factor models, however, are only applicable to the flow algorithms such as `StokesFlow`.
:::